# Métodos Montecarlo Fing 2022 - Entrega 1

**Autor:** Carlos M. Martinez, marzo-abril 2022.

**Email:** carlosm@fing.edu.uy, carlos@cagnazzo.uy



Supongamos que para construir una casa debemos efectuar la siguiente lista de tareas:

- T1 - cimientos - tiempo aleatorio uniforme entre 40 y 56 hs. 
- T2 - contrapiso - tiempo aleatorio uniforme entre 24 y 32 hs. 
- T3 - paredes - tiempo aleatorio uniforme entre 20 y 40 hs.
- T4 - techo - tiempo aleatorio uniforme entre 16 y 48 hs.
- T5 - instalación sanitaria - tiempo aleatorio uniforme entre 10 y 30 hs.
- T6 - instalación el eléctrica - tiempo aleatorio uniforme entre 15 y 30 hs. 
- T7 - cerramientos - tiempo aleatorio uniforme entre 20 y 25 hs.
- T8 - pintura - tiempo aleatorio uniforme entre 30 y 50 hs.
- T9 - revestimientos sanitarios - tiempo aleatorio uniforme entre 40 y 60 hs.
- T10 - limpieza final - tiempo aleatorio uniforme entre 8 y 16 hs.

Hay ciertas dependencias que implican que una tarea no puede comenzar hasta haberse terminado otra previa:

- T2, T3 dependen de 1.
- T4 depende de 2 y 3
- T5 depende de 2 y 3
- T6 depende de 3
- T7 depende de 3
- T8 dependede 4,5,6 y7
- T9 depende de 5
- T10 depende de 7, 8 y 9

## Entrega 1 

**Ejercicio 2.1**

Implementar un programa que reciba como par ́ametros de l ́ınea de comando (o pregunte en pantalla) la cantidad de replicaciones n a realizar, y emplee Monte Carlo para calcular (e imprimir) la estimación del tiempo total promedio desde que se comienza la obra hasta que se finaliza la misma, y la desviación estándar de este estimador.

In [1]:
import math
import matplotlib as mpl
import matplotlib.pyplot as plt
import random
from pathos.multiprocessing import ProcessPool as Pool
from time import perf_counter

# Estructura de datos para las tareas:

TareasDB = ['']*11

TareasDB[1] = {
    'dur': (40,56), # intervalo de duración en horas
    'dep': []       # tareas de la que depende para poder comenzar
}

TareasDB[2] = {
    'dur': (24,32), # intervalo de duración en horas
    'dep': [1]       # tareas de la que depende para poder comenzar
}

TareasDB[3] = {
    'dur': (20,40), # intervalo de duración en horas
    'dep': [1]       # tareas de la que depende para poder comenzar
}

TareasDB[4] = {
    'dur': (16,48), # intervalo de duración en horas
    'dep': [2,3]       # tareas de la que depende para poder comenzar
}

TareasDB[5] = {
    'dur': (10,30), # intervalo de duración en horas
    'dep': [2,3]       # tareas de la que depende para poder comenzar
}

TareasDB[6] = {
    'dur': (15,30), # intervalo de duración en horas
    'dep': [3]       # tareas de la que depende para poder comenzar
}

TareasDB[7] = {
    'dur': (20,25), # intervalo de duración en horas
    'dep': [3]       # tareas de la que depende para poder comenzar
}

TareasDB[8] = {
    'dur': (30,50),        # intervalo de duración en horas
    'dep': [4,5,6,7]       # tareas de la que depende para poder comenzar
}

TareasDB[9] = {
    'dur': (40,60),        # intervalo de duración en horas
    'dep': [5]       # tareas de la que depende para poder comenzar
}

TareasDB[10] = {
    'dur': (8,16),        # intervalo de duración en horas
    'dep': [7,8,9]       # tareas de la que depende para poder comenzar
}


In [2]:
# Generar muestras de las duraciones de las tareas

def sampleTSim(wTareasDB, n):
    TSim = [0] * 11
    random.seed()
    for i in range(1, n):
        TSim[i] = random.randint(wTareasDB[i]['dur'][0], wTareasDB[i]['dur'][1])
    return TSim
## 
    
sampleTSim(TareasDB,11)

[0, 54, 32, 35, 48, 26, 16, 20, 48, 41, 12]

In [3]:
# Calculo del tiempo de construcción

# Alg1 , "ir para atras"
# Tomo la tarea final
# Le sumo la duración max(tareas pre condicion)
# repito para la tarea que tenía ese max

def backtrackTareas(wTSim, wTDB, n):
    # print("corriendo con n: ",n)
    
    if n == 1:
        return [1]
    else:
        t_max_precond = 0
        max_precond = 0
        for j in wTDB[n]['dep']:
            if wTSim[j] > max_precond: 
                max_precond = wTSim[j]
                t_max_precond = j
            # end if
        # end for
        # print(j)
        prev = backtrackTareas(wTSim, wTDB, t_max_precond)
        prev.append(n)
        return prev
# end def

def calculoTiempoDeConstruccion(lista_tareas, wTSim):
    """
    listaTareas es un array con las tareas en el orden en que se ejecutaron
    """
    dur = 0
    for x in lista_tareas:
        dur = dur + wTSim[x]
    # end for
    return dur
# end def

def sortearX():
    ts = sampleTSim(TareasDB,11)
    tareas = backtrackTareas(ts, TareasDB, 10)
    return calculoTiempoDeConstruccion(tareas, ts)

In [4]:
muestras_promedio = []
muestras_devstd = []

def EstimacionMonteCarlo(n, FsortearX):
    t0 = perf_counter()
    random.seed()
    Xest = 0
    Vest = 0
    for x in range(n):
        Xi = FsortearX()
        Xest = Xest + Xi
        Vest = Vest + Xi**2
    # end for
    
    Xestt = Xest
    Vestt = Vest
    
    Xest = Xest / n
    Vest = Vest/(n*(n-1)) - Xest**2/(n-1)
    
    return (Xest, Vest, Xestt, Vestt, perf_counter()-t0)
# end def



In [5]:
EstimacionMonteCarlo(10**4, sortearX)

(164.271, 0.013062782178218324, 1642710, 271155762, 0.29503488899999963)

In [8]:
def EstimacionMonteCarloParalelo(N, FsortearX, hilos):
    """
        version paralelizada del montecarlo
        N: numero de muestras
        FsortearX: funcion que implementa el experimento
        hilos: cantidad de hilos en el pool de tareas
    """
    t0 = perf_counter()

    args1 = []
    args2 = [] 
    for x in range(0,hilos):
        args1.append( math.ceil(N/hilos) )
        args2.append(FsortearX)
    
    p = Pool(hilos)
    resultados = p.map(EstimacionMonteCarlo, args1, args2 )

    # unir los resultados para producir el resultado final
    Xtotal = 0
    Ntotal = 0
    Vtotal = 0
    
    for i in range(0, hilos):
        Xtotal = Xtotal + resultados[i][2]
        Vtotal = Vtotal + resultados[i][3]        
        Ntotal = Ntotal + math.ceil(N/hilos)
    # end for
    
    #
    Xest = Xtotal / Ntotal
    Vest = Vtotal/(Ntotal*(Ntotal-1)) - Xest**2/(Ntotal-1)
    #
    return (Xest, Vest, Ntotal,  perf_counter()-t0)
# end estimacion montecarlo paralelo

## Resultados

### Corremos la simulación para diferentes tamaños de muestra

In [12]:
import pandas as pd
import numpy as np

resultados = pd.DataFrame(
    {
        'n': [],
        'prom': [],
        'var': [],
        'devstd': [],
        't_run': []
    }
)


tam = [10, 10**2, 10**3, 10**4, 10**5, 10**6, 10**7]

for n in tam:
    random.seed()
    (promedio, v, _, t) = EstimacionMonteCarloParalelo(n, sortearX,8)
    devstd = math.sqrt( v*n)
    
    resultados.loc[len(resultados.index)] = [n,promedio,v,devstd,t]
# end for

resultados

,n,prom,var,devstd,t_run
0,10.0,168.375000,10.173958,10.086604,0.007505
1,100.0,164.038462,1.039381,10.195002,0.005796
2,1000.0,164.394000,0.126071,11.228127,0.015035
3,10000.0,164.581700,0.012772,11.301517,0.124332
4,100000.0,164.321890,0.001311,11.450732,1.073778
5,1000000.0,164.360738,0.000129,11.373203,10.411825
6,10000000.0,164.359666,0.000013,11.373152,121.726875


## Discusión de los resultados

La varianza del estimador se hace muy pequeña rápidamente, lo que sugiere que la precision con la que estamos estimando la esperanza de la V.A. del tiempo de construcción es muy buena.

En Python los tiempos de ejecución se vuelven bastante elevados para tamaños de muestra no tan grandes. En gran medida esto se debe a que Python por defecto es single-threaded.

## Datos adicionales y referencias

### Información acerca del software y hardware utilizados

**Software:**
 - Python 3.8.10 corriendo en Windows WSL2 (Windows Subsystem for Linux)
 - Jupyter Notebook 
 
**Librerias:** 
 - pandas
 - numpy
 
**Hardware:**

 - PC Windows 11, con WSL2
 - CPU Intel Core i5 10400F (6 cores)
 - 16 GB de RAM